In [1]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

  0%|          | 49152/9912422 [00:00<00:20, 484124.08it/s]

9920512it [00:00, 38651274.61it/s]                           


Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


32768it [00:00, 1436113.70it/s]
1654784it [00:00, 12896289.89it/s]                         

Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw
Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



8192it [00:00, 176349.39it/s]


Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw
Processing...
Done!


In [3]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [4]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)

In [5]:
model = Net()

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

In [15]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [16]:
for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 1.803073
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.675048
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.639521
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.525580
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.450598
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.505832
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.480697
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.411785
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.234784
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.311302
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.270222
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.172785
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.123831
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.015301
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.052050
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.017929
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.949874
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.891793
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.834537
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.965265
Train Epoch: 1 [12800/60000 (